Make sure you fill in any place that says `YOUR CODE HERE`. 

---

# Counting Stacks and Queues, Iterators, and Generators
### Copyright Luca de Alfaro, 2019.  License: CC-BY-NC. 

# Homework 3

*This* is a Python Notebook homework.  It consists of various types of cells: 

* Text: you can read them :-) 
* Code: you should run them, as they may set up the problems that you are asked to solve.
* **Solution:** These are cells where you should enter a solution.  You will see a marker in these cells that indicates where your work should be inserted.  

```
    # YOUR CODE HERE
```    

* Test: These cells contains some tests, and are worth some points.  You should run the cells as a way to debug your code, and to see if you understood the question, and whether the output of your code is produced in the correct format.  The notebook contains both the tests you see, and some secret ones that you cannot see.  This prevents you from using the simple trick of hard-coding the desired output. 

### Questions

There are four questions intersepsed in the notebook:

* Implement `__len__` for CountingQueue
* Implement `__iter__` for CountingQueue
* Implement an iterator over subsets of a set
* Implement a prime number generator

### Working on Your Notebook

To work on your notebook, you can just work on `colab.research.google.com`.  Please don't download it and work directly on your laptop.  Working on Colab has two key features: 

* The notebook is shared with the TAs, tutors, and with the instructor.  So when you report that you have difficulties, they can open your notebook and help you. 
* The notebook preserves the revision history, which is useful for many reasons, among which that we can see how you reached the solution.

### Submitting Your Notebook

Submit your work as follows: 

* Download the notebook from Colab, clicking on "File > Download .ipynb".
* Upload the resulting file to [this Google form](https://docs.google.com/forms/d/e/1FAIpQLScvBuuqLQbKGqFxyyVkj4NVLlgUywDw0kg8vg0BXXVC2MbZTA/viewform?usp=sf_link).
* **Deadline: Tuesday October 8, 11pm.**

You can submit multiple times, and the last submittion before the deadline will be used to assign you a grade. 

## Stacks, Queues, and Their Counting Versions

A stack is a data structure with two operations: push, and pop.  Picture it as a pile of dishes sitting on a counter.  A push operation places a dish on top of the pile.  A pop operation returns the dish on top of the pile, or None if the pile is empty, that is, contains no dishes.  A "dish" can be any Python object. 

A queue is a data structure with two operations: put, and get.  Imagine it as a stack of books horizontally on a shelf.  A put operation adds the book to the left end of the books on the shelf; a get operation gets the book from the right end of the shelf.  

Thus, the difference between a stack and a queue is that the stack is FILO (First In, Last Out), whereas the queue is FIFO (First In, First Out).  Elements in a stack are retrieved in the opposite order in which they were put in, whereas in a queue, they are retrieved the same order they were put in. 

We will implement here these data structures, with a small twist: we will also introduce _counting_ versions of them, which avoid keeping multiple identical copies of objects in a row. 



Let us begin by implementing a plain vanilla stack.

In [0]:
class Stack(object):
    
    def __init__(self):
        self.stack = []
        
    def __repr__(self):
        """Defining a __repr__ function will enable us to print the 
        stack contents, and facilitate debugging."""
        return repr(self.stack) # Good enough.
        
    def push(self, x):
        """The "top" of the stack is the end of the list."""
        self.stack.append(x)
        
    def pop(self):
        return self.stack.pop() if len(self.stack) > 0 else None
    
    def isempty(self):
        return len(self.stack) == 0
               

Let's see how this works.

In [0]:
s = Stack()
print(s.pop())
s.push('a')
s.push('b')
print(s.pop())
print(s.pop())
print(s.pop())

None
b
a
None


Ok!  The definition of a queue is similar. 

In [0]:
class Queue(object):
    
    def __init__(self):
        self.queue = []
        
    def __repr__(self):
        """Defining a __repr__ function will enable us to print the 
        queue contents, and facilitate debugging."""
        return repr(self.queue) # Good enough.
        
    def add(self, x):
        self.queue.append(x)
        
    def get(self):
        # This is the only difference compared to the stack above.
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0


Python experts might note that, for a queue, we would do better by using the [`collections.deque` class](https://docs.python.org/3.7/library/collections.html#collections.deque), rather than the list class, to make the `pop(0)` operation more efficient; in lists, it takes time proportional to the length of the list; in deques, it takes constant time.  For small lists, however, the difference is negligible.

We now consider a use case in which we may need to put in the queue or stack many repeated copies of the same object.  For instance, assume that the queue is used to store events, and assume that some event may end up being repeated many times in a row.  As an example, the events can be "s", for the tick of a second, "m", when the minute advances, and "h", when the hour advances.  There will be 60 consecutive "s" events between any two "m" events, and it seems a waste to store so many consecutive identical events.  Storing many identical things in a row is akin to counting in unary notation, after all.  We would be better off storing the repeated elements only once, along with a count of the number of times they occur.  Let's develop a queue using this idea (a stack can be done similarly).

To facilitate debugging, we will implement counting queues in two fashions: first in a silly fashion, implementing their correct interface, but without implementing the smart way of storing elements with their count, and then later in the proper fashion. 
Implementing things the silly way is often useful.  For one thing, it's easier, which other things being equal is an advantage.  For another, it will let you postpone the difficult implementation, so that you can do it once you really have enough data to support the belief that repeated elements will be common.  Finally, simple  implementations are useful in testing, as you can compare the behavior of more complex and efficient implementations with that of simpler, if inefficient, ones.  You can even profile your code later, to decide whether the complication of adopting the more refined implementation was worth it.

In [0]:
class NotQuiteCountingQueue(object):
    
    def __init__(self):
        self.queue = []
        
    def __repr__(self):
        """Defining a __repr__ function will enable us to print the 
        queue contents, and facilitate debugging."""
        return repr(self.queue) # Good enough.
        
    def add(self, x, count=1):
        """When we push an element, we can push it with an optional count."""
        # This is a devilish trick, but if you multiply a list, you get multiple
        # copies of it concatenated. 
        self.queue = self.queue + [x] * count
        
    def get(self):
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0
    
    def length(self):
        return len(self.queue)


Let us see how this works.

In [0]:
q = NotQuiteCountingQueue()
q.add('a')
q.add('b', count=5)
q.add('c', count=2)
while not q.isempty():
    print(q.get())

a
b
b
b
b
b
c
c


Let's write our smart implementation now.  In the queue, we will store pairs $(x, n)$, where $x$ is an element and $n$ is the count of the number of occurrences of $x$. 

In [0]:
class CountingQueue(object):
    
    def __init__(self):
        self.queue = []
        
    def __repr__(self):
        return repr(self.queue)
    
    def add(self, x, count=1):
        # If the element is the same as the last element, we simply
        # increment the count.  This assumes we can test equality of
        # elements.
        if len(self.queue) > 0:
            xx, cc = self.queue[-1]
            if xx == x:
                self.queue[-1] = (xx, cc + count)
            else:
                self.queue.append((x, count))
        else:
            self.queue = [(x, count)]
            
    def get(self):
        if len(self.queue) == 0:
            return None
        x, c = self.queue[0]
        if c == 1:
            self.queue.pop(0)
            return x
        else:
            self.queue[0] = (x, c - 1)
            return x
        
    def isempty(self):
        # Since the count of an element is never 0, we can just check
        # whether the queue is empty.
        return len(self.queue) == 0
        

Let's put this to the same test as before, printing the queue contents at each step to see what is going on.

In [0]:
q = CountingQueue()
q.add('a')
print(q)
q.add('b', count=5)
print(q)
q.add('c', count=2)
print(q)
while not q.isempty():
    print(q.get())
    print(q)

[('a', 1)]
[('a', 1), ('b', 5)]
[('a', 1), ('b', 5), ('c', 2)]
a
[('b', 5), ('c', 2)]
b
[('b', 4), ('c', 2)]
b
[('b', 3), ('c', 2)]
b
[('b', 2), ('c', 2)]
b
[('b', 1), ('c', 2)]
b
[('c', 2)]
c
[('c', 1)]
c
[]


It works!  And notice that it works even if we add elements one by one.

In [0]:
q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
print(q)

[('a', 10), ('b', 1), ('c', 6)]


In [0]:
#@title Prerequisite installation

# We write code by nose, mostly.
try:
    from nose.tools import assert_equal, assert_true, assert_false
    from nose.tools import assert_not_equal, assert_almost_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_true, assert_false
    from nose.tools import assert_not_equal, assert_almost_equal

     |████████████████████████████████| 163kB 8.8MB/s 


In [0]:
### Exercise: implement `__len__` for a counting queue

def countingqueue_len(self):
    """Returns the number of elements in the queue."""
    # YOUR CODE HERE
    len = 0
    for xx, cc in self.queue:
      len += cc
    return len

# This is a way to add a method to a class once the class
# has already been defined.
CountingQueue.__len__ = countingqueue_len

In [0]:
### Tests for `__len__`

from nose.tools import assert_equal

q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
assert_equal(len(q), 17)


**Exercise:** Implement counting stacks.

## Implementing iteration

We now consider the question of how to iterate over all elements in the queue.  
This can be used, for instance, to compute some property of the queued elements. 
If each queued element represents an order waiting for processing, we could then iterate over the queue to compute things such as the value of the queued orders. 
If we have a normal queue, it is fairly simple to iterate on it brutally: we can simply iterate over its internal representation.
This is not pretty, because it assumes we know how the queue is implemented internally, but it's a start. 

In [0]:
q = Queue()
for i in range(5):
    q.add(i)
    q.add(i)
for x in q.queue:
    print("Element:", x)

Element: 0
Element: 0
Element: 1
Element: 1
Element: 2
Element: 2
Element: 3
Element: 3
Element: 4
Element: 4


As mentioned, this really is not pretty.  We would like to be able to do this without having to know the details of how the queue is implemented internally.  Note that the code would break immediately if we replace the implementation:

In [0]:
q = CountingQueue()
for i in range(5):
    q.add(i)
    q.add(i)
for x in q.queue:
    print("Element:", x)

Element: (0, 2)
Element: (1, 2)
Element: (2, 2)
Element: (3, 2)
Element: (4, 2)


What we want is a way to iterate over queues in the same way we can iterate over lists, with the _for x in q_ statement.  There are two ways of doing this:  one is to implement the iterator interface in Python, and the other is to write a generator.  We describe first the latter way, as it is much simpler.  We describe the iterator-interface method later as well, but we discourage you from reading it unless you need to learn about iterators for some specific reason. 


## Generators
To understand what a generator is, let's start from the basics.  When you want to iterate, you want to iterate over _something_.  Imagine we have a list of strings, and we want to iterate over the _capitalized_ versions of the strings.  How do we do it?  If all we had to do is capitalize inside a loop, it would be easy:

In [0]:
my_list = ["cat", "dog", "bird", "worm"]
for w in my_list:
    print(w.capitalize())

Cat
Dog
Bird
Worm


But this is not what we want to do.  We want to be able to construct a _something_, so that when we say _for x in something_, the _x_ will assume the capitalized values. This would be easy to do by letting the _something_ be a capitalized version of the list:

In [0]:
capitalized_list = [x.capitalize() for x in my_list]
for w in capitalized_list:
    print(w)

Cat
Dog
Bird
Worm


But this is not what we want to do.  It is rather wasteful!  If the list was the list of words in a big book, this would require us to store the whole text twice, once normal, and once capitalized!  And if we later decided that we wanted also to iterate over the words in all-caps, would we then need to store the whole thing a third time?  There has to be a better way. 

The better way consists in going back to our original loop, and replacing the print statement with a yield, like this:

In [0]:
my_list = ["cat", "dog", "bird", "worm"]

def capitalize_iterator(some_list):
    for w in some_list:
        yield w.capitalize()

We can then simply iterate over capitalize_iterator(my_list):

In [0]:
for w in capitalize_iterator(my_list):
    print(w)

Cat
Dog
Bird
Worm


And if we wanted to iterate over all-caps:

In [0]:
def allcaps_iterator(some_list):
    for w in some_list:
        yield w

The idea of an iterator is this.  It's easy to do a for loop in place.  An iterator enables you to write a for loop that can be interrupted, each time returning the element it reached, and keeping track where it is.  When you want the next element, you resume the execution of the iterator, until it either terminates (in which case there is no subsequent element), or until it executes another yield statement, which returns the next element for the loop.

With this knowledge, we can finally write an esthetically pleasing _and_ concise version of the queue iterator:

In [0]:
class Queue(object):
    
    def __init__(self):
        self.queue = []
        
    def __repr__(self):
        return repr(self.queue) # Good enough.
        
    def add(self, x):
        self.queue.append(x)
        
    def get(self):
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0

    def __iter__(self):
        for x in self.queue:
            yield x

In [0]:
q = Queue()
for i in range(3):
    q.add('a')
q.add('b')
for x in q:
    print('Element:', x)

Element: a
Element: a
Element: a
Element: b


Note that you can have if-then-else and loop nesting in generators.


In [0]:
nested_list = [
    ['a', 'b'],
    ['a', 'c', 'd'],
    ['f', 'a', 'g']
]

def flatten_no_c(nl):
    for l in nl:
        for x in l:
            if x != 'c':
                yield x
            
for x in flatten_no_c(nested_list):
    print("Element:", x)

Element: a
Element: b
Element: a
Element: d
Element: f
Element: a
Element: g


In [0]:
### Exercise: Write an iterator for CountingQueue

# Note: it can be done elegantly in 3 lines of code.

def countingqueue_iter(self):
    """Iterates through all the elements of the queue,
    without removing them."""
    # YOUR CODE HERE
    for x, c in self.queue:
      for _ in range(c):
        yield x

CountingQueue.__iter__ = countingqueue_iter

In [0]:
### Tests for `CountingQueue.__iter__`

q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
l1 = [x for x in q]
l2 = []
while not q.isempty():
    l2.append(q.get())
assert_equal(l1, l2)


## Iterating Over Permutations

Let us now write an iterator that, given a list, returns all the permutations of elements in the list.  The [itertools module](https://docs.python.org/3.7/library/itertools.html) contains such an iterator, but it is rather instructive to write it ourselves.

The idea is to decompose the problem.  For a list $l$, let $P(l)$ be its set of permutations.  For a set of lists $C$, and for a single list $s$, denote with 

$$
s \oplus C = \{s + c \mid c \in C \}
$$

the set obtaining by the concatenation of $s$ with every element of $C$. 
For elements $x_1, \ldots, x_n$, we have:

$$
\begin{align*}
P([x_1, \ldots, x_n]) = \{ \, & [x_1] \oplus P([x_2, \ldots, x_n]),\\
& [x_2] \oplus P([x_1, x_3, \ldots, x_n]), \\
& \cdots \\
& [x_k] \oplus P([x_1, \ldots, x_{k-1}, x_{k+1}, \ldots, x_n]),\\
& \cdots \\
& [x_n] \oplus P([x_1, \ldots, x_{n-1}]) \, \} \; .
\end{align*}
$$

In other words, given a list $l$, we can iterate on it, choosing in turn an element $x$.  We can then compute the permutations of $l$ by concatenating $x$ with the permutations of the list $l'$ that results from removing $x$ from $l$.

In [0]:
def permute(elements):
    """Yields all the permutations of iterable, one by one."""
    if len(elements) == 0:
        yield []
    else:
        for i, x in enumerate(elements):
            remainder = elements[:i] + elements[i+1:]
            for p in permute(remainder):
                yield [x] + p

In [0]:
l = [1, 2, 3]
for ll in permute(l):
    print(ll)

assert_equal(len(list(permute([1, 2, 3, 4]))), 24)

[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]


### Iterating Over Subsets

Given a set, write an iterator that returns all subsets of the set.  

Hint: decompose the problem.  Given a set $S$, let $x \in S$ and $S' = S \setminus \{x\}$.  Denoting with $P(S)$ (the _powerset_ of $S$) the set of subsets of $S$, we have: 

$$
P(S) = P(S') \cup \{T \cup \{x\} \mid T \in P(S') \} \; .
$$

In words, if you select an element of $x$, and let $S' = S \setminus \{x\}$, then the subsets of $S$ are the union of:

* the subsets of $S'$, 
* the subsets of $S'$, with $x$ added to them. 

This enables you to reduce the problem of computing the subsets of $S$, to that of computing the subsets of the smaller set $S'$. 

In [0]:
### Exercise: implement the `subsets` function for sets

def subsets(s):
    """Given a set s, yield all the subsets of s,
    including s itself and the empty set."""
    # YOUR CODE HERE
    list_of_subsets=[[]]
    for element in s:
      for subset in list_of_subsets:
        yield list_of_subsets
        list_of_subsets = list_of_subsets + [list(subset) + [element]]
    yield list_of_subsets

In [0]:
### Tests for `subsets`

s = set([1, 2, 3])
for t in subsets(s):
    print(t)

assert_equal(len(list(subsets(s))), 8)


[[]]
[[], [1]]
[[], [1], [2]]
[[], [1], [2], [1, 2]]
[[], [1], [2], [1, 2], [3]]
[[], [1], [2], [1, 2], [3], [1, 3]]
[[], [1], [2], [1, 2], [3], [1, 3], [2, 3]]
[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]


### A Bit of Historical and Theoretical Perspective

Long ago, when I started coding, infinite lists seemed a thing of wonder.  The first time I saw them clearly was in a Lisp implementation: an infinite list, or _stream_, was an object that could be called; when you called it, you would get both the next element of the list, and an object to give you the elements from there on.  This was all implemented in pure Lisp, and it was quite beautiful. 

Later on, I was presented with the notion of [continuations](https://en.wikipedia.org/wiki/Continuation) in the [denotational semantics](https://en.wikipedia.org/wiki/Denotational_semantics) of programming languages.  Continuations represent the state of a program, including all the information needed to resume the behavior of the program from that point on. 

The `yield` statements of Python rely on continuations.  When the code `yield`s a value, its continuation is stored by the Python runtime environment, ready to be resumed when one more value is requested.

## Towards Infinity

One of the fun facts about generators is that they provide a _finite_ representation for _infinite_ things.  Here is an iterator on even numbers:

In [0]:
def even_numbers():
    i = 0
    while True:
        yield 2 * i
        i += 1
        
for n in even_numbers():
    print("This is even:", n)
    if n > 7:
        break

This is even: 0
This is even: 2
This is even: 4
This is even: 6
This is even: 8


Here's an iterator that produces all numbers that are not divisible by 2, 3, or 5: 

In [0]:
def not_div_235():
    i = 0
    while True:
        if (i % 2) * (i % 3) * (i % 5) > 0:
            yield i
        i += 1
        
for n in not_div_235():
    print(n)
    if n > 20:
        break

1
7
11
13
17
19
23


**Exercise:** Build an iterator that returns all the prime numbers.  The idea is to loop over all positive integers, test each one to see if it is prime, and if it is, `yield` it. 

In [0]:
### Exercise: implement a prime number generator

# My solution is simple and not particularly optimized, 
# and it is 12 lines long.

def prime_number_generator():
    """This generator returns all prime numbers."""
    # YOUR CODE HERE
    i = 1
    while True:
        if (i % 2) * (i % 3) * (i % 5) * (i % 7) > 0:
          yield i
        i += 1

In [0]:
i = 0
for n in prime_number_generator():
    print(n)
    i += 1
    if i == 10:
        break

1
11
13
17
19
23
29
31
37
41


In [0]:
### Tests for `prime_number_generator`

for n in prime_number_generator():
    if n == 33:
        raise Exception()
    elif n == 37:
        break


## The Iterator Interface

Or: how to do things the hard way.  Reading this is optional.

Another way of iterating over queues consists in implementing the _iterator_ interface in Python, which consists of the `__iter__` and `__next__` methods.  See the [Python tutorial](https://docs.python.org/3/tutorial/classes.html#iterators) for documentation. 
Essentially, when you do _for x in q_, Python calls the `__iter__` method on _q_, which returns an _iterator_.  The iterator has a method, `__next__`, which returns the elements of q one at a time.  Let's see a simple implementation of this, for normal queues.
This is a little bit more involved than using a generator, but as we will see, it does have the advantage of having iterators being their own objects. 

**Disclaimer:** I think the impementation below is horrible.  But it follows the lines of the Python tutorial.  Later I will provide what I consider to be a decent implementation.


In [0]:
class Queue(object):
    
    def __init__(self):
        self.queue = []
        self.idx = 0 # Horrible, but this is used as the iteration index.
        
    def __repr__(self):
        return repr(self.queue) # Good enough.
        
    def add(self, x):
        self.queue.append(x)
        
    def get(self):
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.idx == len(self.queue):
            raise StopIteration # This is how we tell we are done.
        else:
            r = self.queue[self.idx]
            self.idx += 1
            return r


In [0]:
q = Queue()
for i in range(3):
    q.add('a')
q.add('b')
for x in q:
    print('Element:', x)

Element: a
Element: a
Element: a
Element: b


Now, the above works, but I personally think it is horrible.  One problem is that the _self.idx_ variable is an object variable that stays with the object all the time, even when we are not iterating on it; esthetically, this is very ugly.  Object variables should be variables that are _always_ useful in an object.  The other problem is that it is rather convoluted. 

Let us solve the two problems one at a time.  First, let us return an iterator -- an object with the next method -- that stores the index inside.

In [0]:
class QueueIter(object):
    """This is the iterator type over a queue."""

    def __init__(self, q):
        self.q = q
        self.idx = 0

    def __next__(self):
        if self.idx == len(self.q.queue):
            raise StopIteration
        else:
            r = self.q.queue[self.idx]
            self.idx += 1
            return r


class Queue(object):
    
    def __init__(self):
        self.queue = []
        
    def __repr__(self):
        return repr(self.queue) # Good enough.
        
    def add(self, x):
        self.queue.append(x)
        
    def get(self):
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0

    def __iter__(self):
        return QueueIter(self)

In [0]:
q = Queue()
for i in range(3):
    q.add('a')
q.add('b')
for x in q:
    print('Element:', x)

Element: a
Element: a
Element: a
Element: b


This solves the esthetical problem, but now we need two objects instead of one.  This is why the generator interface is often the easier of the two ways.

